In [1]:
from sqlalchemy import create_engine

In [2]:
import pandas as pd

In [3]:
HOST = 'localhost'
PORT = '5432'
USERNAME = '*******'
PASSWORD = '*******' 
DB = 'movies'

In [4]:
conn_string = f'postgres://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}'

conn_string_mac = f'postgres://{HOST}:{PORT}/{DB}'

In [5]:
conn_string_mac

'postgres://localhost:5432/movies'

In [6]:
engine = create_engine(conn_string)

In [7]:
query = """
SELECT * FROM links;
"""

In [8]:
engine.execute(query)

In [9]:
df = pd.read_sql(query, engine)
print(df.head())
df_links=df.copy()
df_links.head()

   movieid  imdbid   tmdbid
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0


,movieid,imdbid,tmdbid
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [10]:
df_links.describe()

,movieid,imdbid,tmdbid
count,9742.000000,9.742000e+03,9734.000000
mean,42200.353623,6.771839e+05,55162.123793
std,52160.494854,1.107228e+06,93653.481487
min,1.000000,4.170000e+02,2.000000
25%,3248.250000,9.518075e+04,9665.500000
50%,7300.000000,1.672605e+05,16529.000000
75%,76232.000000,8.055685e+05,44205.750000
max,193609.000000,8.391976e+06,525662.000000


In [58]:
query = """
SELECT * FROM movies;
"""

In [59]:
df = pd.read_sql(query, engine)
df_movies=df.copy()
df_movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
df_movies.describe()

,movie_id
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


In [14]:
query = """
SELECT * FROM ratings;
"""

In [15]:
df = pd.read_sql(query, engine)
df_ratings=df.copy()
df_ratings.head()

,userid,movieid,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [16]:
df_ratings.describe()

,userid,movieid,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [17]:
query = """
SELECT * FROM tags;
"""

In [18]:
df = pd.read_sql(query, engine)
df_tags=df.copy()
df_tags.head()

,userid,movieid,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


# NMF on Movie Data

11.2. Non-Negative Matrix Factorization

In [19]:
import numpy as np
from sklearn.decomposition import NMF

In [20]:
data = [df_ratings['rating']]

In [21]:
data

[0         4.0
 1         4.0
 2         4.0
 3         5.0
 4         5.0
          ... 
 100831    4.0
 100832    5.0
 100833    5.0
 100834    5.0
 100835    3.0
 Name: rating, Length: 100836, dtype: float64]

In [22]:
# movie, ratings by users
#data = [df_ratings['rating']]
#columns = [df_ratings['movieid']] #movies
#index = [df_ratings['userid']] #users

#need a dataframe for this
#R = pd.DataFrame(data, index=index, columns=columns).values
df_ratings.head()
df_ratings.set_index('userid', inplace=True)

In [23]:
#df_ratings.head()
df_ratings.drop('timestamp',1, inplace=True)

In [25]:
df_ratings.reset_index(inplace=True)

In [38]:
R = df_ratings.pivot(index='userid', columns='movieid', values='rating')

In [39]:
R

movieid,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userid,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
from sklearn.impute import KNNImputer

In [29]:
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

In [30]:
imputer.fit(R)

KNNImputer()

In [31]:
R_trans = imputer.transform(R)

In [32]:
R_trans.shape

(610, 9724)

In [34]:
R.reset_index(inplace=True)

In [35]:
R

movieid,userid,1,2,3,4,5,6,7,8,9,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,609,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
R_t=R.T

In [37]:
R_t.index

Index(['userid',        1,        2,        3,        4,        5,        6,
              7,        8,        9,
       ...
         193565,   193567,   193571,   193573,   193579,   193581,   193583,
         193585,   193587,   193609],
      dtype='object', name='movieid', length=9725)

In [40]:
index=R.index
column=R.columns

In [41]:
df_r_trans=pd.DataFrame(data=R_trans, index=index, columns=column)

In [97]:
df_r_trans.to_csv (r'C:\Users\denni\Desktop\spiced_academy\challenges_dennis\Week_10\recommender_module\flask_app\R.csv',index = False, header=True)


In [98]:
R=pd.read_csv('R.csv')

In [99]:
R

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,4.0,3.6,4.0,2.8,3.4,4.0,3.2,3.0,2.9,3.6,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
1,4.0,3.6,2.9,2.1,2.2,3.7,2.8,3.0,2.4,3.3,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
2,3.7,3.0,2.1,2.7,2.8,3.3,2.5,2.8,2.8,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
3,3.9,3.6,3.6,2.5,3.2,3.2,3.6,3.0,3.4,3.7,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
4,4.0,3.4,3.3,2.1,2.6,4.0,3.2,2.6,3.5,3.5,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.5,3.6,3.3,2.7,2.8,3.8,2.5,2.4,2.6,3.9,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
606,4.0,3.8,4.0,2.8,3.6,3.8,4.0,2.6,3.3,3.9,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
607,2.5,2.0,2.0,2.1,2.9,4.3,3.2,2.8,2.6,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
608,3.0,3.3,3.5,2.5,3.1,3.9,3.3,2.8,2.5,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0


In [41]:
df_r_trans

movieid,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userid,,,,,,,,,,,,,,,,,,,,,
1,4.0,3.6,4.0,2.8,3.4,4.0,3.2,3.0,2.9,3.6,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
2,4.0,3.6,2.9,2.1,2.2,3.7,2.8,3.0,2.4,3.3,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
3,3.7,3.0,2.1,2.7,2.8,3.3,2.5,2.8,2.8,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
4,3.9,3.6,3.6,2.5,3.2,3.2,3.6,3.0,3.4,3.7,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
5,4.0,3.4,3.3,2.1,2.6,4.0,3.2,2.6,3.5,3.5,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,3.6,3.3,2.7,2.8,3.8,2.5,2.4,2.6,3.9,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
607,4.0,3.8,4.0,2.8,3.6,3.8,4.0,2.6,3.3,3.9,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
608,2.5,2.0,2.0,2.1,2.9,4.3,3.2,2.8,2.6,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0


In [42]:

#R=df_r_trans
#create a model and set the hyperparameters
# model assumes R ~ PQ'
model = NMF(n_components=20, init='random', random_state=10)

model.fit(df_r_trans)

Q = model.components_  # movie-feature matrix

P = model.transform(df_r_trans)  # user-feature matrix

print(model.reconstruction_err_) #reconstruction error

nR = np.dot(P, Q)
print(nR) ## The reconstructed matrix!

# predict the hidden features for a new data point
#query = [[1, 2, 5, 4, 5]]
#in this case, a new user providing ratings for the 5 movies.
#print(model.transform(query))

c:\users\denni\anaconda3\envs\flaskenv\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


425.5764869073012
[[4.38870528 3.86911469 3.60182837 ... 3.50405247 3.50540656 4.00600413]
 [4.01941248 3.40683955 3.38018625 ... 3.50601733 3.4967309  3.99340728]
 [3.30628353 2.87048597 2.60289806 ... 3.49569156 3.50288087 4.00106389]
 ...
 [2.9841892  3.05762229 2.62182362 ... 3.48803795 3.50383762 3.99081991]
 [3.66324086 3.15070044 3.02531012 ... 3.49886123 3.49988443 4.00157576]
 [4.3847028  3.59345772 3.18110846 ... 3.51093165 3.48864289 3.97240268]]


c:\users\denni\anaconda3\envs\flaskenv\lib\site-packages\sklearn\decomposition\_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


In [48]:
nR.shape

(610, 9724)

To map movie names to movie ID’s, construct a {name: id} dictionary

In [49]:
#df_movies.set_index("movie_id", drop=True, inplace=True)
df_movies
#df_movies.drop('genres', inplace=True)

,movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [50]:
movie_dict=df_movies.to_dict('index') 

In [52]:
movie_dict[0]

{'movie_id': 1,
 'title': 'Toy Story (1995)',
 'genres': 'Adventure|Animation|Children|Comedy|Fantasy'}

Create an vector of three movies the user likes. Set these to 5 and all others to zero.

In [53]:
new_user= np.zeros((1,9721))
new_user=np.insert(new_user,(1,3,5),5).reshape(1,-1)
#model.transform(new_user)
new_user

array([[0., 5., 0., ..., 0., 0., 0.]])

In [54]:
new_user.shape

(1, 9724)

In [55]:
nR.shape

(610, 9724)

In [56]:
recommendation=np.dot(model.transform(new_user),Q)

In [62]:
recommendation.shape

(1, 9724)

In [57]:
result = np.where(recommendation == np.amax(recommendation))
#maxElement = np.amax(recommendation)
result

(array([0], dtype=int64), array([1485], dtype=int64))

In [60]:
movie_dict[1485]

{'movie_id': 2010, 'title': 'Metropolis (1927)', 'genres': 'Drama|Sci-Fi'}

In [43]:
#export model nmf
import pickle
with open('nmf_model.bin', 'wb') as file:
    pickle.dump(model, file)

In [92]:
import pandas as pd

# Extrahieren der Filmtitel
with open('movie_titles', 'w') as file:
    for movie_title in df_final_titles.columns:
        file.write(movie_title)
        file.write('\n')

In [60]:
df_titles=df_movies

In [61]:
df_titles.columns

Index(['movie_id', 'title', 'genres'], dtype='object')

In [62]:
df_titles.drop(columns=['genres'], inplace=True)

In [65]:
df_titles_id=df_titles.set_index('movie_id')

In [66]:
df_titles_id

,title
movie_id,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)
...,...
193581,Black Butler: Book of the Atlantic (2017)
193583,No Game No Life: Zero (2017)
193585,Flint (2017)


In [70]:
df_titles_id.to_csv(r'C:\Users\denni\Desktop\spiced_academy\challenges_dennis\Week_10\recommender_module\flask_app\movietitle_db_id.csv', index=True, header=True)

In [71]:
movie_title=pd.read_csv('movietitle_db_id.csv')

In [72]:
movie_title

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017)
9738,193583,No Game No Life: Zero (2017)
9739,193585,Flint (2017)
9740,193587,Bungo Stray Dogs: Dead Apple (2018)


In [ ]:
movie_title['title'].where(movie_title['movie_id'] == final_recommendations.index)

In [84]:
movie_title.set_index('movie_id').shape#.iloc[2075,0]

(9742, 1)

In [ ]:
final_recommendations = recommendations.T[boolean_mask].sort_values('new_user').iloc[-1:]
    final_recommendations_t=final_recommendations.T
    #for i in range len(final_recommendations.index):
    movie_title_t=movie_title.set_index('movie_id')#.T
    return movie_title_t['title'].where(movie_title['movie_id'] == final_recommendations_t.index)

In [50]:
R=pd.read_csv('R.csv')

In [51]:
R

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
0,4.0,3.6,4.0,2.8,3.4,4.0,3.2,3.0,2.9,3.6,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
1,4.0,3.6,2.9,2.1,2.2,3.7,2.8,3.0,2.4,3.3,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
2,3.7,3.0,2.1,2.7,2.8,3.3,2.5,2.8,2.8,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
3,3.9,3.6,3.6,2.5,3.2,3.2,3.6,3.0,3.4,3.7,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
4,4.0,3.4,3.3,2.1,2.6,4.0,3.2,2.6,3.5,3.5,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,2.5,3.6,3.3,2.7,2.8,3.8,2.5,2.4,2.6,3.9,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
606,4.0,3.8,4.0,2.8,3.6,3.8,4.0,2.6,3.3,3.9,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
607,2.5,2.0,2.0,2.1,2.9,4.3,3.2,2.8,2.6,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
608,3.0,3.3,3.5,2.5,3.1,3.9,3.3,2.8,2.5,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
